## Statistical analysis of electricity consumption

Q: How much could be saved by using electricity during the cheapest time periods vs during the peak hours?

Assumptions:
- most of the consumption of the household is done during the cheapest times

Need:
- fetch the historic price information from the tibber api
- Average household electricity consumption per day (inc. charging a Tesla Model 3 every other day)

Estimate:
- What is the cumulative price difference between consuming most of the electricity during cheap hours vs peak hours over a year?

In [22]:
import sys
sys.path.append('/Users/andreas/fun/useful_py_func/pyTibber/tibber/')
#sys.path.append('/home/andram/fun/LED-env/lib/python3.11/site-packages/tibber')
from tibber import Tibber, gql_queries
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import asyncio
import numpy as np


In [29]:
from tibber import Tibber, gql_queries
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np

#TODO: Think more about how to perform the statistical analysis on the consumption vs max/min price
#What is the question I'm trying to answer?
#If the consumption during the peak hours could be moved to the cheaper hours, how much could be saved?

async def main():
    TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
    USER_AGENT = "LED_client"
    HOME_ID = "975996b6-e7ca-4fbf-9f72-61e2df95bc0c"

    tibber_connection = Tibber(TOKEN, user_agent=USER_AGENT)
    await tibber_connection.update_info()
    
    data = await fetch_data(tibber_connection, HOME_ID)

    d = daily_data(data)
    daily_savings = calculate_potential_savings(d)
    print(f"Average daily potential savings: {daily_savings}")


async def fetch_data(tibber_connection, home_id):
    query = f"""{{
      viewer {{
        home(id: "{home_id}") {{
          consumption(resolution: HOURLY, last: 8760, before: "MjAyNC0wMi0yM1QxODowMDowMC4wMDArMDE6MDA=") {{
            pageInfo {{
              startCursor
            }}
            nodes {{
              from
              cost
              consumption
            }}
          }}
        }}
      }}
    }}"""
    data = await tibber_connection.execute(query)
    return data


def daily_data(data):
    # Extract dates, costs, and consumptions
    dates = [datetime.strptime(node['from'], '%Y-%m-%dT%H:%M:%S.%f%z') for node in data['viewer']['home']['consumption']['nodes']]
    costs = [node['cost'] for node in data['viewer']['home']['consumption']['nodes']]
    consumptions = [node['consumption'] for node in data['viewer']['home']['consumption']['nodes']]
    
    # Group by day
    daily_data = {}
    for date, cost, consumption in zip(dates, costs, consumptions):
        day = date.strftime('%Y-%m-%d')
        if day not in daily_data:
            daily_data[day] = []
        daily_data[day].append((cost, consumption))
    
    return daily_data

def calculate_potential_savings(daily_data):
    total_savings = []

    for day, data in daily_data.items():
        prices, consumptions = zip(*data)
        average_price = np.mean(prices)
        peak_price_threshold = average_price * 1.10
        cheap_price_threshold = average_price

        peak_hours_consumption = sum(consumption for price, consumption in data if price > peak_price_threshold)
        cheap_hour_price = min(prices)

        # Assume all peak consumption can be shifted to the cheapest hour of the day
        daily_saving = (peak_price_threshold - cheap_hour_price) * peak_hours_consumption
        total_savings.append(daily_saving)

    return np.mean(total_savings)


await main()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x116a52840>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x11193f110>, 146536.412401916)]']
connector: <aiohttp.connector.TCPConnector object at 0x116a53740>


Average daily potential savings: 18.56825897219829
